In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.model_selection import KFold, train_test_split
import itertools
from sklearn.feature_extraction import DictVectorizer
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score

In [2]:
""" 
Step 1: read in data file.

Columns should be named as the following: 

1. mpg: continuous 
2. cylinders: multi-valued discrete 
3. displacement: continuous 
4. horsepower: continuous 
5. weight: continuous 
6. acceleration: continuous 
7. model year: multi-valued discrete 
8. origin: multi-valued discrete 
9. car name: string (unique for each instance)

"""

read=pd.read_csv('auto_mpg.data', delim_whitespace = True,
               names = ["mpg", "cylinders", "displacement", "horsepower", "weight", "accerleration", "model year", "origin", "car name"]) 
                 
data = read.drop(['car name'], axis=1, inplace = False)
print(data.shape)
data.head(8)


(398, 8)


,mpg,cylinders,displacement,horsepower,weight,accerleration,model year,origin
0,18.0,8,307.0,130.0,3504.0,12.0,70,1
1,15.0,8,350.0,165.0,3693.0,11.5,70,1
2,18.0,8,318.0,150.0,3436.0,11.0,70,1
3,16.0,8,304.0,150.0,3433.0,12.0,70,1
4,17.0,8,302.0,140.0,3449.0,10.5,70,1
5,15.0,8,429.0,198.0,4341.0,10.0,70,1
6,14.0,8,454.0,220.0,4354.0,9.0,70,1
7,14.0,8,440.0,215.0,4312.0,8.5,70,1


In [3]:
"""Step 2: Clean the data

1). Noted that in the original dataset, categorical column "origin" is stored as numerical, I need to convert them into string. 
2). Check datatype.
3). Drop NANs and other data that does not make sense. 

"""

# Convert 'origin' column into string:
data['origin'] = data['origin'].astype(str)

# Clean data: check first if there is any NANs.
print(data.isnull().any()) # No NAN. If there is any, comment out the following. 
#df = data.dropna(how='all')
print()

col = list(data.columns.values)
print("Columns names: \n{}".format(col))
print()

# Check each column's type: noted that horsepower is str. 
for column in col:
    print("Type for column {}: {}".format(column, type(data[column][0])))
    
print()
    
# Check the string: I found '?' in there, which cannot be converted to numeirc. 
index_list = data['horsepower'].index[data['horsepower'] == '?'].tolist()
print("Index that containing special characters:\n{}".format(index_list))
print()
print(data.shape)
# Drop the rows with index_list generated above. 
df=data.drop(np.array(index_list))

# Need to reset the index, otherwise the index are gone after the drop. 
df = df.reset_index(drop=True)
print(df.shape)




mpg              False
cylinders        False
displacement     False
horsepower       False
weight           False
accerleration    False
model year       False
origin           False
dtype: bool

Columns names: 
['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'accerleration', 'model year', 'origin']

Type for column mpg: <class 'numpy.float64'>
Type for column cylinders: <class 'numpy.int64'>
Type for column displacement: <class 'numpy.float64'>
Type for column horsepower: <class 'str'>
Type for column weight: <class 'numpy.float64'>
Type for column accerleration: <class 'numpy.float64'>
Type for column model year: <class 'numpy.int64'>
Type for column origin: <class 'str'>

Index that containing special characters:
[32, 126, 330, 336, 354, 374]

(398, 8)
(392, 8)


In [4]:
""" Step 3:

1. Rewrite columns into their normalized form;
2. Convert string column to numeric column.

"""

for column in col:
    if column != 'horsepower' and column != 'origin':
        mean = df[column].mean()
        std = df[column].std()
        df[column] = (df[column]-mean)/std
    if column == 'horsepower':
        for ele in range(0,len(df['horsepower'])):
            df[column][ele] = (float(df[column][ele]))


df.head(8)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,mpg,cylinders,displacement,horsepower,weight,accerleration,model year,origin
0,-0.697747,1.482053,1.075915,130,0.619748,-1.283618,-1.623241,1
1,-1.082115,1.482053,1.486832,165,0.842258,-1.464852,-1.623241,1
2,-0.697747,1.482053,1.181033,150,0.539692,-1.646086,-1.623241,1
3,-0.953992,1.482053,1.047246,150,0.536160,-1.283618,-1.623241,1
4,-0.825870,1.482053,1.028134,140,0.554997,-1.827320,-1.623241,1
5,-1.082115,1.482053,2.241772,198,1.605147,-2.008554,-1.623241,1
6,-1.210238,1.482053,2.480677,220,1.620452,-2.371022,-1.623241,1
7,-1.210238,1.482053,2.346890,215,1.571005,-2.552256,-1.623241,1


In [5]:
""" Question 3: a. Clean the data, removing samples with empty entries and scaling each feature to have zero mean and unit variance

Step 4: Rewrite horsepower into its standardized form.

Centralize the data by subtracting the mean, then devide the data with standard deviation. 

"""

mean = df['horsepower'].mean()
std = df['horsepower'].std()
df['horsepower'] = (df['horsepower']-mean)/std

df.head(8)

,mpg,cylinders,displacement,horsepower,weight,accerleration,model year,origin
0,-0.697747,1.482053,1.075915,0.663285,0.619748,-1.283618,-1.623241,1
1,-1.082115,1.482053,1.486832,1.57258,0.842258,-1.464852,-1.623241,1
2,-0.697747,1.482053,1.181033,1.18288,0.539692,-1.646086,-1.623241,1
3,-0.953992,1.482053,1.047246,1.18288,0.536160,-1.283618,-1.623241,1
4,-0.825870,1.482053,1.028134,0.923085,0.554997,-1.827320,-1.623241,1
5,-1.082115,1.482053,2.241772,2.42992,1.605147,-2.008554,-1.623241,1
6,-1.210238,1.482053,2.480677,3.00148,1.620452,-2.371022,-1.623241,1
7,-1.210238,1.482053,2.346890,2.87158,1.571005,-2.552256,-1.623241,1


In [6]:
""" Step 5: 

i.  DictVectorization for categorical features, aka 'origins'. 
ii. Fit the model with LiearRegression() to get the best model for all features. 

Noted that I used the train_test_split here to split up the dataset, the test size is set to be 10% of the total data.
In such way I expected to make it more comparable with the 10-fold CV. 

"""
np.random.seed(0)

# Feature columns:
col = ['cylinders', 'displacement', 'horsepower', 'weight', 'accerleration', 'model year', 'origin']

# Create original X and y:
Original_X = df[col]
Original_y = df['mpg']

# Dict_Vectorize the X: 
dict_data = Original_X.T.to_dict().values()

# Initiate vectorizer
vectorizer = DictVectorizer(sparse=False)

# Transform X and y: y does not need to transform. 
X = vectorizer.fit_transform(dict_data)
y = Original_y

# Use tran_test_split from sklearn to split the test and train. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10)

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# The coefficients
print('Coefficients: \n', regr.coef_)

# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
print()

# Sum of squared error, aka residual sum of squares. 
print("Sum of squared error, aka residual sum of squares.: %.2f"
      % ((mean_squared_error(y_test, y_pred))*len(y_test)))
print()

#print("Residual Sum of squares: %.2f" % np.mean((y_pred - y_test) ** 2))

# Explained variance score: a result of 1 denotes a perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))


Coefficients: 
 [ 0.046677   -0.08986849  0.30446999 -0.06381986  0.35855809 -0.22516793
  0.09657048  0.12859745 -0.7553884 ]
Mean squared error: 0.17

Sum of squared error, aka residual sum of squares.: 6.82

Variance score: 0.82


In [7]:
""" Calculate the Akaike Information Criterion (AIC) and Bayesian Information Criterion (BIC) for two models
– one with all coefficients  

Step 6: AIC and BIC for all features. Should also be with all data.

"""

# Fit all data:
# Create original X and y:
Original_X = df[col]
Original_y = df['mpg']

# Dict_Vectorize the X: 
dict_data = Original_X.T.to_dict().values()

# Initiate vectorizer
vectorizer = DictVectorizer(sparse=False)

# Transform X and y: y does not need to transform. 
X = vectorizer.fit_transform(dict_data)
y = Original_y

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X, y)

# Make predictions using the testing set
y_pred = regr.predict(X)


k = 7+1
n = len(y)

mean_sq_er_total = mean_squared_error(y, y_pred)

# AIC and BIC for All coefficients
aic_total = (2*k) + n * np.log(mean_sq_er_total)
print("AIC for all coefficients is:\n{} ".format(aic_total))
bic_total = k * np.log(n) + n * np.log(mean_sq_er_total)
print("BIC for all coefficients is:\n{} ".format(bic_total))
print("*******************************************")
print()

AIC for all coefficients is:
-666.4561468340506 
BIC for all coefficients is:
-634.686052115727 
*******************************************



In [8]:
""" Calculate the Akaike Information Criterion (AIC) and Bayesian Information Criterion (BIC) for two models
– one with one coefficient  

Step 7: AIC and BIC for one feature. These are statistical methods, so I fitted the model with all samples. 

From above I know that "weight" has the most significant magnitute in coefficient.
Then X will be weight in this question. 

"""

np.random.seed(0)


# Independent variable dataframe X.
# X = df['accerleration'][:, np.newaxis]
X = df['accerleration'][:, np.newaxis]

y = df['mpg'][:, np.newaxis]

# Use tran_test_split from sklearn to split the test and train. 
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

# Create linear regression object
regr = linear_model.LinearRegression()

# Fill the model using all samples
regr.fit(X, y)

# Make predictions using the testing set
y_pred = regr.predict(X)

# The coefficients
print('Coefficients: \n', regr.coef_)

# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y, y_pred))
print()

# Sum of squared error, aka residual sum of squares. 
print("Sum of squared error, aka residual sum of squares.: %.2f"
      % ((mean_squared_error(y, y_pred))*len(y_test)))
print()

# Explained variance score: a result of 1 denotes a perfect prediction
print('Variance score: %.2f' % r2_score(y, y_pred))
print()

# BIC and AIC: 
k = 1+1
n = len(y)

mean_sq_er_total = mean_squared_error(y, y_pred)

# AIC and BIC for All coefficients
aic_total = (2*k) + n * np.log(mean_sq_er_total)
print("AIC for all coefficients is:\n{} ".format(aic_total))
bic_total = k * np.log(n) + n * np.log(mean_sq_er_total)
print("BIC for all coefficients is:\n{} ".format(bic_total))
print("*******************************************")
print()

Coefficients: 
 [[ 0.42332854]]
Mean squared error: 0.82

Sum of squared error, aka residual sum of squares.: 32.75

Variance score: 0.18

AIC for all coefficients is:
-74.41516012147483 
BIC for all coefficients is:
-66.4726364418939 
*******************************************



In [9]:
""" Step 8: Find a better model based on AIC and BIC. 

1. Define a function to iterate through given numbers of independent variable,
and return variable combinations. 
   a). I set 'weight' as the main independent variable, since it has the highest coefficient.
   b). To simplify the problem, I only use 2 and 6 features, which means to select 1 and 5 variables from list, since there is already a 'weight'.

"""

def models(size):
    # Define other independent variables for me to choose from. 
    Ind = ["cylinders", "displacement", "horsepower", "accerleration", "model year", "origin"]
    # Create the combinations for chosen number of variables. 
    lst_select = list(itertools.combinations(Ind,size))
    print("Model Selected:")
    print()

    # Create the final model selection list. 
    lst_models = []
    for ele in lst_select:
        lst_main = ["weight"]
        lst_main.extend(ele)
        lst_models.append(lst_main)


    print()
    return lst_models




In [10]:
""" Step 8: Find a better model based on AIC and BIC. 

Fit the model with all instances. 

2. Define a function to perform linear regression and caculate the AIC and BIC.
   a). I set 'weight' as the main independent variable, since it has the highest coefficient.
   b). To simplify the problem, I only use 2 and 5 features, which means to select 1 and 4 variables from list, since there is already a 'weight'.

"""

def Linear_IC(model):
    for m in model:
        Original_X = df[m]
        Original_y = df['mpg']
        
        # Dict_Vectorize the X: 
        dict_data = Original_X.T.to_dict().values()

        # Initiate vectorizer
        vectorizer = DictVectorizer(sparse=False)

        # Transform X and y: y does not need to transform. 
        X = vectorizer.fit_transform(dict_data)
        # y does not need to be vectorized, since this is a numerical column.
        y = Original_y
        # Use tran_test_split from sklearn to split the test and train. 
        # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10)

        # Create linear regression object
        regr = linear_model.LinearRegression()

        # Train the model using the training sets
        regr.fit(X, y)

        # Make predictions using the testing set
        y_pred = regr.predict(X)
        
        k = len(model)+1
        n = len(y)

        mean_sq_er_total = mean_squared_error(y, y_pred)

        # AIC and BIC for All coefficients
        aic_total = (2*k) + n * np.log(mean_sq_er_total)
        print("AIC for {} is:\n{} ".format(m, aic_total))
        bic_total = k * np.log(n) + n * np.log(mean_sq_er_total)
        print("BIC for {} is:\n{} ".format(m, bic_total))
        print("*******************************************")
        print()    
    

In [11]:
""" Step 9. 10-fold cross validation

Define a class CrossValidation(), and inside the class, define the function k_fold(k, model_list):
k - number of folds;
model_list: the list containing the column names for the selected features. 

Noted that this class should only be used after the dataframe is defined, cleaned and named as 'df'.

"""

class CrossValidation_Manually():
    
    def __init__(self, model_list): 
        self.inst_attr = "Selected Features for Cross Validation." 
        self.model_list = model_list
    
    def __repr__(self):
        return(print("Selected features: \n{}".format(self.model_list)))

    
    def K_fold(self):
        
        self.R2_Manually = []
        self.MSE_Manually = []
        
        for i in range(10):
            print("****************Fold {} as the test dataset*******************".format(i))

            # Noted that before execute the function, I need the data after the extra datapoints taken out. 
            # Namely, I have df_new and df_target_new. 
            n = len(df_target_new)
            array = np.arange(n).reshape((n, 1))
            #print(array)

            split = np.vsplit(array, 10)

            # Create linear regression object
            regr = linear_model.LinearRegression()
        
            X_train = np.delete(np.array(df_new), (split[i].tolist()), axis=0)
            
            X_step_1 = np.array(df_new)[split[i].tolist(),]
            X_test = np.squeeze(X_step_1)

            y_train = np.delete(np.array(df_target_new), (split[i].tolist()), axis=0)
        
            y_step_1 = np.array(df_target_new)[split[i].tolist(),]  
            y_test = np.squeeze(y_step_1)
    
            regr.fit(X_train, y_train)
    
            y_pred = regr.predict(X_test)
    
            print('Coefficients: \n', regr.coef_)

            # The mean squared error
            print("Mean squared error: %.2f"
                  % mean_squared_error(y_test, y_pred))
            self.MSE_Manually.append(mean_squared_error(y_test, y_pred))
    
            # Explained variance score: a result of 1 denotes a perfect prediction
            print('Variance score: %.2f' % r2_score(y_test, y_pred))
            self.R2_Manually.append(r2_score(y_test, y_pred))
    
            print()

    # For further questions in logistic regression.
    def K_fold_logreg(self):
        
        #self.R2_logreg = []
        self.MSE_logreg = []
        self.score_logreg = []
        self.accuracy_logreg = []
        
        for i in range(10):
            print("****************Fold {} as the test dataset*******************".format(i))

            # Noted that before execute the function, I need the data after the extra datapoints taken out. 
            # Namely, I have df_new and df_target_new. 
            n = len(df_target_new)
            array = np.arange(n).reshape((n, 1))
            #print(array)

            split = np.vsplit(array, 10)
            
            # Train and test data:
            X_train = np.delete(np.array(df_new), (split[i].tolist()), axis=0)
            
            X_step_1 = np.array(df_new)[split[i].tolist(),]
            X_test = np.squeeze(X_step_1)

            y_train = np.delete(np.array(df_target_new), (split[i].tolist()), axis=0)
        
            y_step_1 = np.array(df_target_new)[split[i].tolist(),]  
            y_test = np.squeeze(y_step_1)
    
            # Initiate logistic regression model:
            logreg = linear_model.LogisticRegression(C=1e5)
            
            logreg.fit(X_train, y_train)

            y_pred = logreg.predict(X_test)

            #print("Prediction: {}".format(y_pred))

            print("Logistic Regression Score with LogisticRegression.scoore(): {}".format(logreg.score(X_test, y_test)))
            self.score_logreg.append(logreg.score(X_test, y_test))
            print()
            
            print("Logistic Regression Score with Metrics.accuracy_score(): {}".format(accuracy_score(y_test, y_pred)))
            self.accuracy_logreg.append(accuracy_score(y_test, y_pred))
            print()

In [12]:
""" Use 10-fold cross validation and MSE as a metric to select among these three  models. Manually Method.

Model 1: ['weight', 'cylinders', 'displacement', 'horsepower', 'model year', 'origin'] since the AIC and BIC are the smallest, suggesting a good fit:

Step 1: Since I am going to use 10-folds manually, I need to drop extra data points with the Victorized matrix.

"""

Original_X = df[['weight', 'cylinders', 'displacement', 'horsepower', 'model year', 'origin']]
Original_y = df['mpg']
# Dict_Vectorize the X: 
dict_data = Original_X.T.to_dict().values()

# Initiate vectorizer
vectorizer = DictVectorizer(sparse=False)

# Transform X and y: y does not need to transform. 
X = pd.DataFrame(vectorizer.fit_transform(dict_data))
y = Original_y

print("To determine how many data points to be droped, I need the shape of vectorized data:\n{}".format(X.shape))

np.random.seed(10)

# Set the number of samples to be removed.
remove_n = 2

# Randomly choose remove_n number of index from dataframe.
drop = np.random.choice(df.index, remove_n, replace=False)

# Drop rows that are randomly chosen based on indexes, for fearure data.
df_new = X.drop(drop)

# Drop the same rows in target data.
# Write target into panda dataframe for easier furthur process.
df_target_new = y.drop(drop)

print("The shape of new data:\n{}".format(df_new.shape))

To determine how many data points to be droped, I need the shape of vectorized data:
(392, 8)
The shape of new data:
(390, 8)


In [13]:
""" Use 10-fold cross validation and MSE as a metric to select among these three  models. Manually Method.

Model 1: ['weight', 'cylinders', 'displacement', 'horsepower', 'model year', 'origin'] since the AIC and BIC are the smallest, suggesting a good fit:

Step 2: Use the Class CrossValidation_Manually for ['weight', 'cylinders', 'displacement', 'horsepower', 'model year', 'origin'].

"""

CV = CrossValidation_Manually(['weight', 'cylinders', 'displacement', 'horsepower', 'model year', 'origin'])
CV.K_fold()
#print(CV.R2_Manually)
print("Mean R2 for Manually generated 10-fold:\n{}".format(np.mean(CV.R2_Manually)))
print()
#print(CV.MSE_Manually)
print("Mean MSE for Manually generated 10-fold:\n{}".format(np.mean(CV.MSE_Manually)))

****************Fold 0 as the test dataset*******************
Coefficients: 
 [-0.10787796  0.33716095 -0.185793    0.40050197 -0.24389737  0.10558049
  0.13831689 -0.68280928]
Mean squared error: 0.16
Variance score: 0.65

****************Fold 1 as the test dataset*******************
Coefficients: 
 [-0.13991014  0.31472152 -0.10714193  0.38044561 -0.2249957   0.10093149
  0.12406422 -0.71582277]
Mean squared error: 0.15
Variance score: 0.77

****************Fold 2 as the test dataset*******************
Coefficients: 
 [-0.10415255  0.29695365 -0.14149993  0.34373634 -0.22552212  0.07906398
  0.14645814 -0.71160246]
Mean squared error: 0.16
Variance score: 0.66

****************Fold 3 as the test dataset*******************
Coefficients: 
 [-0.14413236  0.38894516 -0.14398435  0.35870711 -0.25047262  0.12718146
  0.12329117 -0.72871381]
Mean squared error: 0.15
Variance score: 0.76

****************Fold 4 as the test dataset*******************
Coefficients: 
 [-0.118021    0.32459559 -